### New implementation of fitter and rotations ?

In [3]:
# Imports
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from tetris_main import *
import random
from library.hill_climbing.hc import hill_climb
from library.simulated_annealing.sa import simulated_annealing
from library.genetic_algorithm import gao

In [4]:
T_DICT = {
    "I": [(0, 0), (1, 0), (2, 0), (3, 0)],
    "L": [(0, 0), (1, 0), (2, 0), (2, 1)],
    "J": [(0, 1), (1, 1), (2, 1), (2, 0)],
    "T": [(0, 1), (1, 0), (1, 1), (1, 2)],
    "S": [(0, 1), (1, 0), (0, 2), (1, 1)],
    "Z": [(0, 0), (0, 1), (1, 1), (1, 2)],
    "O": [(0, 0), (1, 0), (0, 1), (1, 1)],
}
def get_piece_coordinates(piece: str):

    piece_type = piece[0]
    c = np.array(T_DICT[piece_type])
    rotation = int(piece[1])

    if rotation == 1:
        c = np.flip(c, axis=1)
        c[:, 1] *= -1
        # back to positives - shift
        c[:, 1] += np.abs(np.min(c[:, 1]))
        
    elif rotation == 2:
        c *= -1
        # back to positives - shift
        c+=abs(c.min())
        
    elif rotation == 3:
        c = np.flip(c, axis=1)
        c[:, 0] *= -1
        # back to positives - shift
        c[:, 0] += np.abs(np.min(c[:, 0]))
        
    # push to upper left if needed
    if 0 not in c[:, 0]: 
        c[:, 0]-=np.min(c[:, 0])
    if 0 not in c[:, 1]: 
        c[:, 1]-=np.min(c[:, 1])
        
    return c

In [5]:
#get_piece_coordinates("J2")

In [9]:
def tetrimino_fitter(pieces: list, grid_shape: tuple, verbose: bool = False) -> tuple:
    """Fits all pieces into a grid with a given shape.

    Args:
        pieces (list): List with piece type and rotation as str
        grid_shape (tuple): (x, y)
        verbose (bool, optional): Defaults to False.

    Returns:
        (tuple): Filled grid, pieces' coordinates
    """
    # Create pieces_coordinates list
    pieces_coordinates = []

    # Initialize grid
    grid = np.zeros(grid_shape)
    # Transform the corresponding piece into its matrix representation.
    # Consider rotation and how that impacts the piece, and apply the corresponding transformation here.
    # Identify spots that can be filled.
    free_space_array = np.column_stack(
        np.where(grid == 0)
    )  # this can be used to find the coordinates of all the zeros, and have them stored.
    for coord in free_space_array:
        free_space_array2 = np.column_stack(
            np.where(grid == 1)
            ) 
        if verbose:
            print(free_space_array2)
        if (free_space_array2 == coord).all(-1).any():
            if verbose:
                print(coord,"skipping")
            continue
        if verbose:
            print(f"First zero in sequence found at {tuple(coord)}")
        if verbose: print(pieces)

        for each in pieces:
            piece = get_piece_coordinates(each)

            # get array for piece and displace it by the coordinates
            p = piece + coord
            if verbose: print(p)

            # check that piece is not out of bounds
            if not (
                (max(p[:, 0]) > (grid.shape[0] - 1))
                or (max(p[:, 1]) > (grid.shape[1] - 1))
            ):
                if verbose: print('its not out of bounds')
                # Get the test fit
                test_fit = np.array([grid[x, y] for x, y in p])
                if verbose: print(test_fit)
                # If the sum is 0, it fits
                if np.sum(test_fit) == 0:
                    if verbose: print("fill")
                    # Fill up the spaces with 1
                    for x, y in p:
                        grid[x, y] = 1
                    # Add the piece's coordinates to the list
                    pieces_coordinates.append(p)
                    # We placed the piece so we stop trying to fit the piece
                    pieces.remove(each)
                    if verbose: print(grid)
                    break
        # perfect match found - all 1's
        if np.sum(grid) == grid.shape[0]*grid.shape[1]:
            break

        
    return grid, pieces_coordinates

In [11]:
# compare it with the previous one :)))
pieces = ["O1","O3","O2","O1"]
grid, pieces_coordinates = tetrimino_fitter(pieces, (4,4),verbose=True)

[]
First zero in sequence found at (0, 0)
['O1', 'O3', 'O2', 'O1']
[[0 1]
 [0 0]
 [1 1]
 [1 0]]
its not out of bounds
[0. 0. 0. 0.]
fill
[[1. 1. 0. 0.]
 [1. 1. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
[[0 0]
 [0 1]
 [1 0]
 [1 1]]
[0 1] skipping
[[0 0]
 [0 1]
 [1 0]
 [1 1]]
First zero in sequence found at (0, 2)
['O3', 'O2', 'O1']
[[1 2]
 [1 3]
 [0 2]
 [0 3]]
its not out of bounds
[0. 0. 0. 0.]
fill
[[1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
[[0 0]
 [0 1]
 [0 2]
 [0 3]
 [1 0]
 [1 1]
 [1 2]
 [1 3]]
[0 3] skipping
[[0 0]
 [0 1]
 [0 2]
 [0 3]
 [1 0]
 [1 1]
 [1 2]
 [1 3]]
[1 0] skipping
[[0 0]
 [0 1]
 [0 2]
 [0 3]
 [1 0]
 [1 1]
 [1 2]
 [1 3]]
[1 1] skipping
[[0 0]
 [0 1]
 [0 2]
 [0 3]
 [1 0]
 [1 1]
 [1 2]
 [1 3]]
[1 2] skipping
[[0 0]
 [0 1]
 [0 2]
 [0 3]
 [1 0]
 [1 1]
 [1 2]
 [1 3]]
[1 3] skipping
[[0 0]
 [0 1]
 [0 2]
 [0 3]
 [1 0]
 [1 1]
 [1 2]
 [1 3]]
First zero in sequence found at (2, 0)
['O2', 'O1']
[[3 1]
 [2 1]
 [3 0]
 [2 0]]
its not out of bounds
[0. 0. 0. 0.]
fill
[